In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from PIL import Image 
from selenium.webdriver.common.action_chains import ActionChains
import requests
from bs4 import BeautifulSoup as bs
import os
import time
import io
from random import randint
from time import sleep
import re
import datetime
import time

def get_cach(driver):
    while True:
        driver.get('http://search.anccnet.com/searchResult2.aspx')   
        driver.fullscreen_window()
        sleep(2)

        element = driver.find_element_by_class_name("l-captcha")
        element.click()
        sleep(2)
        driver.save_screenshot('tem.png',)

        #######
        im = Image.open('tem.png') 


        im = im.resize((1440,900))

        # 图片的宽度和高度
        img_size = im.size
        print('图片识别')

        x = 720
        y = 154
        w = 300
        h = 160
        test = im.crop((x, y, x+w, y+h))
        width = test.size[0]
        height = test.size[1]
        test = test.convert('RGB')
        array = []
        for x in range(width):
            for y in range(height):
                r, g, b = test.getpixel((x,y))
                rgb = (r, g, b)
                if sum(rgb) == 765:
                    array.append((x,y))
        x_set = list()
        y_set = list()
        for line in array:
            x_set.append(line[0])
            y_set.append(line[1])
        my_x = int(sum(x_set) / float(len(x_set)))
        my_y = int(sum(y_set) / float(len(y_set)))
    #     print(my_x,my_y)
        test.load()[my_x,my_y] = (0,0,0)


        element = driver.find_elements_by_xpath("//iframe")[1]
        print(element.location)

        sleep(1)
        action = ActionChains(driver)
        action.move_to_element_with_offset( element, my_x,  my_y)
        # sleep(1)
        action.click()
    #     sleep(1)
#         action.click()
        action.perform()

        sleep(10)
        driver.refresh()
        res = bs(driver.page_source)
        if len(res.select('#Label1')) != 0:
            print('验证成功')
            break
        else:
            print('验证失败')
            driver.refresh()
            continue
    return driver,test


def  create_driver():
    option = webdriver.ChromeOptions()
    option.arguments.clear()
    for key in headers:
        mystring = key+'=\"'+headers[key]+"\""
        option.add_argument(mystring)
    option.add_argument('--headless')
    ip =  get_new_ip()['http']
    my_proxy ='--proxy-server=' + ip
    
    option.add_argument(my_proxy) 
    option.add_argument("--window-size=1440,900")
    driver = webdriver.Chrome('../test/chromedriver',chrome_options=option)
    driver.set_window_rect(0,0,1440,900)
    driver.delete_all_cookies()

    
    return driver,option,ip

headers = {

'Host':'search.anccnet.com',
'Referer':'http://search.anccnet.com/searchResult2.aspx',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.84 Safari/537.36',
'Upgrade-Insecure-Requests':'1',
'Accept-Encoding':'gzip, deflate',
    
'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',

'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.5,en;q=0.3',

'DNT':'1',
'Connection':'keep-alive',
 'Cookies':'cuuid_322149=46e1c054ee1090e033563512a08143d9'
    
}


def get_new_ip ():
    ip = requests.get("http://http.tiqu.qingjuhe.cn/getip?num=1&type=1&pro=&city=0&yys=0&port=1&pack=23882&ts=0&ys=0&cs=0&lb=1&sb=0&pb=4&mr=0&regions=")
    proxyDict = { 
              "http"  : "http://"+ip.text.strip(), 
              "https" : "https://"+ip.text.strip()
            }
    return proxyDict


def get_product_info(new_res):
    mydic ={}
    mydic['product_price'] = 'null'
    for line in new_res.select('script')[-3].text.split('\n'):   
#         if 'txt_gtin' in line :
#             m = re.search(r"src =(.*)\/\>",line)
#             mydic['product_pic'] = m.group(1)
        if 'Att_Sys_zh-cn_141_G' in line :   
            mydic['product_name'] = re.search(r",\'(.*)\'",line).group(1) 
        if 'Att_Sys_zh-cn_74_G' in line :   
            mydic['product_place'] = re.search(r",\'(.*)\'",line).group(1)  
        if 'Att_Sys_zh-cn_22_G' in line :   
            mydic['product_UNSPSC'] = re.search(r",\'(.*)\'",line).group(1)     
        if 'Att_Sys_zh-cn_332_G' in line :   
            mydic['product_package'] = re.search(r",\'(.*)\'",line).group(1)          
        if 'Att_Sys_zh-cn_196_G' in line :   
            mydic['product_price'] = re.search(r",\'(.*)\'",line).group(1)
    return mydic

In [ ]:
driver.page_source



In [ ]:
import openpyxl
import pandas

# myexcel = pandas.read_excel('test.xlsx',names = ['index','UNSPSC_code','product','package','unit','place'] )
myexcel = pandas.read_excel('千帆掌柜商品数据 (1).xlsx')

link_list = []
for  line in myexcel.iterrows():
    try:
        link_list.append(int(line[1]['条码']))
    except:
        continue

In [3]:
driver,option,ip = create_driver()
driver,test = get_cach(driver)

图片识别
{'x': 720, 'y': 154}
验证成功


/Users/josh_chiu/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/josh_chiu/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [8]:
driver.close()

In [235]:
driver,option,ip = create_driver()
sleep(1)
driver,test = get_cach(driver)

res = bs(driver.page_source)

time_1 = datetime.datetime.now()
driver.add_cookie({'name':driver.get_cookies()[0]['name'],'value':driver.get_cookies()[0]['value']})

start = 1
for keyword in range(0,100):
    time_2 = datetime.datetime.now()
    start +=1
    
    sec = (time_1 - time_2).total_seconds()
    time_1 = time_2
    print(start,link_list[start],str(sec)+'sec')

    
    sleep(1)
    driver.find_element_by_id('keyword').clear()
    search_bar = driver.find_element_by_id('keyword')
    key = link_list[start]
    sleep(1)
    search_bar.send_keys(key)

   
    sleep(2)
    driver.find_element_by_id('gdsBtn').click()
    sleep(1)

    
    res = bs(driver.page_source)


    
    try:
        if '401' in res.select('title')[0].text :
            print(401)
            break
            driver.delete_all_cookies()
            driver.close()
            driver,option,ip = create_driver()
            driver,test = get_cach(driver)
            start-=1
            driver.add_cookie({'name':driver.get_cookies()[0]['name'],'value':driver.get_cookies()[0]['value']})
            continue
    except Exception as e:
        print(e)
        
        
        
        
    
#     if len(res.select('#Label4')) != 0:
#         driver,test = get_cach(driver)
    if   len(res.select('#Label1')) == 0 :
        result[key] = "找不到"  
        print("找不到" )
        sleep(3)
        continue
    if  '已下市' in res.select('#Label1')[0].text :
        result[key] = "已下市"   
        print("已下市" )
        sleep(3)
        continue
    if '暂无相关信息' in res.select('#Label1')[0].text :
        result[key] = "暂无相关信息" 
        print("暂无相关信息" )
        sleep(3)
        continue
    if len(res.select('.p-info a')) == 0:
        result[key] = "奇怪了" 
        print("奇怪了" )
        sleep(3)
        continue
    if "未公开" in (res.select('.p-supplier dd')[-1].text):
        result[key] = "未公开" 
        print("未公开" )
        sleep(3)
        continue
        
    link_click = driver.find_element_by_class_name('p-info')
    sleep(3)
    link_click.find_element_by_tag_name('a').click()  
    driver.switch_to_window(driver.window_handles[1])
    sleep(3)
    new_res = bs(driver.page_source)
    temp_dic = get_product_info(new_res)
    temp_dic['link'] = res.select('.p-info a')[0].text
    temp_dic['brand'] = res.select('.p-supplier dd')[0].text
    temp_dic['company'] = res.select('.p-supplier dd')[1].text
    driver.close()
    driver.switch_to_window(driver.window_handles[0])
    sleep(3)
    
    result[res.select('.p-info a')[0].text] = temp_dic   
    print(start,"正确资料",link_list[start])
    ip_list.append(ip)

图片识别
{'x': 720, 'y': 154}


/Users/josh_chiu/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/josh_chiu/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


验证成功
2 6934665088230 -0.013638sec
2 正确资料 6934665088230
3 6922577752189 -14.615432sec
401


In [220]:
print(driver.get_cookies())
# driver.delete_all_cookies()
# driver.desired_capabilities

[{'domain': 'search.anccnet.com', 'expiry': 2173439494, 'httpOnly': False, 'name': 'ASP.NET_SessionId', 'path': '/', 'secure': False, 'value': 'lwxojdi1gg040455ppwkhzit'}]


In [219]:
driver.execute_script("return localStorage")

[]

In [225]:
driver.find_element_by_id('keyword').clear()
search_bar = driver.find_element_by_id('keyword')

In [226]:
search_bar.send_keys('巧克力')

In [241]:
driver.back()

In [242]:
driver.

'<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml"><head>\n<meta charset="gb2312" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n    <meta name="keywords" content="GDSN, 商品, 产品, 生产企业, 制造商, 零售商, GS1China, 中国物品编码中心" />\n    <meta name="description" content="UNSPSC 联合国标准产品服务与分类代码 分类产品列表 " />\n    <meta name="save" content="history" />\n    \n    <meta http-equiv="Expires" content="0" />\n    <meta http-equiv="Cache-Control" content="no-cache" />\n    <meta http-equiv="Pragma" content="no-cache" />\n    <title>条码数据库，商品信息，生产企业，制造商，零售商，全球数据同步--搜索结果</title>\n    <link rel="stylesheet" href="http://www.anccnet.com/css/common.css" />\n    <link rel="stylesheet" href="http://www.anccnet.com/css/paginationWeb.css" />\n    <link rel="stylesheet" href="http://www.anccnet.com/css/width100.css" />\n<link rel="stylesheet" href="http://www.anccnet.com/V3/style/zCool-1.0/icons.css" />\n<link rel="stylesheet" href="http://www.anccnet.com/V3/style/zCool-1.0/form.css" />\n<li